1. Set Up the Environment

2. Extract Text from PDF Files

In [1]:
import os
import PyPDF2

In [2]:
def extract_text_from_pdfs(pdf_path): 
    """
    Extrait le texte de chaque page des fichiers PDF fournis,
    et combine tout le texte en une seule chaîne.
    """
    all_text = ""

    for pdf_path in pdf_path:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            pdf_text = ""

            for page_num, page in enumerate(reader.pages):
                text = page.extract_text()
                if text:
                    pdf_text += text + "\n"

            print(f" {os.path.basename(pdf_path)} : {len(reader.pages)} pages extraites.")
            all_text += pdf_text + "\n"

    return all_text

In [3]:
# Liste de fichiers PDF à traiter
pdf_files = ["ich-guideline-good-clinical-practice-e6r2.pdf"]

# Extraction du texte
texte_total = extract_text_from_pdfs(pdf_files)

# Aperçu dans le terminal
print("\n Aperçu du texte extrait :\n")
print(texte_total[:1000])  # Affiche les 1000 premiers caractères seulement

 ich-guideline-good-clinical-practice-e6r2.pdf : 68 pages extraites.

 Aperçu du texte extrait :

 
 
30 Churchill Place  ● Canary Wharf ● London E14 5EU ● United Kingdom  
An agency of the European Union     Telephone  +44 (0)20 3660 6000 Facsimile  +44 (0)20 3660 5555  
Send a question via our website  www.ema.europa.eu/contact  
 
 
© European Medicines Agency, 2018. Reproduction is authorised provided the source is acknowledged.  
 
1 December  2016 
EMA/CHMP/ICH/135/1995  
Committee for Human Medicinal Products  
Guideline for good clinical practice E6(R2)  
Step 5 
Adopted by CHMP for release for consultation  23 July 2015  
Start of public consultation   4 August  2015 
End of consultation (dea dline for comments)   3 February  2016 
Final adoption by CHMP  15 December 2016  
Date for coming into effect  14 June 2017  
 
 
  
 
 
 
Guideline for good clinical practice E6(R2)    
EMA/CHMP/ICH/135/1995   Page 2/68 
 
 Document History  
 
First 
Codification  History  Date New 
Co

3. Split the Text into Chunks

In [4]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def split_text_into_chunks(text, chunk_size=1000, chunk_overlap=200):
    """
    Découpe un texte long en chunks de taille définie avec overlap, en utilisant LangChain.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks = splitter.split_text(text)
    print(f" Nombre de chunks créés : {len(chunks)}")
    return chunks

In [7]:
chunks = split_text_into_chunks(texte_total)

# Aperçu des 2 premiers chunks :
for i, chunk in enumerate(chunks[:2]):
    print(f"\n--- Chunk {i+1} ---\n")
    print(chunk[:500])  # afficher les 500 premiers caractères de chaque chunk

✅ Nombre de chunks créés : 191

--- Chunk 1 ---

30 Churchill Place  ● Canary Wharf ● London E14 5EU ● United Kingdom  
An agency of the European Union     Telephone  +44 (0)20 3660 6000 Facsimile  +44 (0)20 3660 5555  
Send a question via our website  www.ema.europa.eu/contact  
 
 
© European Medicines Agency, 2018. Reproduction is authorised provided the source is acknowledged.  
 
1 December  2016 
EMA/CHMP/ICH/135/1995  
Committee for Human Medicinal Products  
Guideline for good clinical practice E6(R2)  
Step 5 
Adopted by CHMP for rele

--- Chunk 2 ---

EMA/CHMP/ICH/135/1995   Page 2/68 
 
 Document History  
 
First 
Codification  History  Date New 
Codification  
November 
2005 
E6 Approval by the CPMP  under Step 3 and release for 
public consultation.  May 1995  E6 
E6 Approval by the CPMP  under Step 4 and released for 
information . July 1996  E6 
Step 5 corrected version  
E6 Approval by the CPMP of Post-Step 4  editorial 
corrections.  July 2002  E6(R1)  
Current E6(R2)

4. Generate Embeddings and Create a a FAISS Vector Store

--> convertir en vecteurs numériques grâce à GoogleGenerativeAIEmbeddings
--> Les stocker dans une base vectorielle FAISS, pour les interroger plus tard

✅ Extraction du texte (Task 2)
✅ Chunking du texte (Task 3)
🟢 Embeddings + FAISS (Task 4)
⬜ Retrieval & QA chain (Task 5)
⬜ Streamlit UI (Task 6)

In [20]:
pip install faiss-cpu langchain google-generativeai langchain_google_genai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# 1.charges un outil d’embedding fourni par Gemini (Google Generative AI)
# Générer les embeddings avec Gemini
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

In [ ]:
# 2. Charger la clé API depuis .env
load_dotenv() # fonction qui charge les variables depuis .env
api_key = os.getenv("GOOGLE_API_KEY") # la valeur de la clé API depuis le fichier .env

In [23]:
# vérification que la clé a bien été récupérée
print("Clé API détectée :", api_key[:6] + "..." if api_key else " Clé non trouvée")

Clé API détectée : AIzaSy...


In [24]:
# 3. Créer l’outil d'embedding Gemini avec la clé
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key
)

In [25]:
# 4. Préparer les chunks : transformer les chunks en objets Document (LangChain attend des objets appelés Document)
documents = [Document(page_content=chunk) for chunk in chunks]

In [27]:
# 5. Générer les embeddings et créer la base FAISS
vector_store = FAISS.from_documents(documents, embeddings) # création d une base d’indexation intelligente : chaque chunk de ton PDF est représenté en embeding dans FAISS.
#  FAISS va :
# Lire chaque Document
# Générer son embedding avec Gemini
# Les stocker dans une base prête à être interrogée
print(" Embeddings générés et vector store créé.")

 Embeddings générés et vector store créé.


In [ ]:
# 6 : Sauvegarder localement la base vectorielle FAISS
vector_store.save_local("faiss_index")
print(" Base FAISS sauvegardée dans le dossier 'faiss_index'")

 Base FAISS sauvegardée dans le dossier 'faiss_index'


In [34]:
# 7 : Recharger plus tard (quand l'app redémarre)
vector_store = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [ ]:
# # Fonction generate_and_save_embeddings(chunks): tout est regroupé sous une focntion
# def generate_and_save_embeddings(chunks, index_path="faiss_index"):
#     #"""
#     #Génére les embeddings pour chaque chunk de texte,
#     #crée une base vectorielle FAISS et la sauvegarde localement.
#     #"""
#     load_dotenv()
#     api_key = os.getenv("GOOGLE_API_KEY")

#     if not api_key:
#         raise ValueError(" Clé API Google Gemini non trouvée dans .env")

#     embeddings = GoogleGenerativeAIEmbeddings(
#         model="models/embedding-001",
#         google_api_key=api_key
#     )
#     documents = [Document(page_content=chunk) for chunk in chunks]

#     vector_store = FAISS.from_documents(documents, embeddings)
#     print(f"✅ {len(documents)} documents vectorisés.")

#     vector_store.save_local(index_path)
#     print(f"✅ Base FAISS sauvegardée dans le dossier '{index_path}'.")

#     return vector_store

Récap du projet pro
✅ PDF → Texte brut
✅ Texte → Chunks
✅ Chunks → Embeddings (Gemini)
✅ Embeddings → FAISS vector store
🟢 FAISS + Gemini → Retrieval QA Chain
⬜ UI interactive (Streamlit)

In [31]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [43]:
def create_qa_chain(vector_store):
    """
    Crée une chaîne de questions/réponses basée sur Gemini + FAISS
    avec un prompt personnalisé.
    """
    
    # 1. Créer le modèle LLM Gemini
    llm = ChatGoogleGenerativeAI(
        model="models/gemini-1.5-pro-001",
        temperature=0.2  # Stable et factuel
    )

    # 2. Créer un prompt template personnalisé
    prompt_template = """
    Tu es un assistant expert en recherche clinique.
    Réponds de façon claire et concise à la question suivante
    uniquement à partir du contexte fourni.

    Si l'information ne se trouve pas dans le contexte, dis : "Je ne sais pas."

    CONTEXTE :
    {context}

    QUESTION :
    {question}
    """

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template
    )

    # 3. Créer la chaîne QA avec LangChain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vector_store.as_retriever(),
        chain_type="stuff",  # Simple, le contexte est injecté tel quel
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=True
    )

    print("✅ Chaîne QA avec Gemini et FAISS prête à l'emploi.")
    return qa_chain

In [44]:
qa_chain = create_qa_chain(vector_store)

# Pose une question sur ton PDF
question = "Quels sont les principes généraux de l'ICH E6 R2 ?"

result = qa_chain.invoke({"query": question})

print("📘 Réponse :\n", result['result'])

✅ Chaîne QA avec Gemini et FAISS prête à l'emploi.
📘 Réponse :
     Les essais cliniques doivent être menés conformément aux principes éthiques qui ont leur origine dans la Déclaration d'Helsinki et qui sont conformes aux BPC et aux exigences réglementaires applicables.


In [38]:
import google.generativeai as genai

In [ ]:
# code pour afficher les modèles dispo :
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

models = genai.list_models()
for m in models:
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking

In [45]:
# Testons une question avec Gemini sur ton document
question = "Quelles sont les responsabilités du promoteur dans l'ICH E6 R2 ?"

In [46]:
result = qa_chain.invoke({"query": question})

In [ ]:
print("Réponse générée par Gemini :\n")
print(result['result'])

📘 Réponse générée par Gemini :

Avant de commencer un essai, le promoteur doit définir, établir et attribuer toutes les tâches et fonctions liées à l'essai. Le promoteur doit également fournir une assurance ou indemniser l'investigateur/l'institution contre les réclamations découlant de l'essai, sauf pour les réclamations qui découlent d'une faute professionnelle et/ou d'une négligence. Le promoteur doit également identifier les risques pour les processus et les données critiques de l'essai et les évaluer. Le promoteur doit décider quels risques réduire et/ou quels risques accepter. 


In [ ]:
# Afficher les sources (chunks) utilisés
print("\n Chunks utilisés comme contexte :")
for i, doc in enumerate(result["source_documents"]):
    print(f"\n--- Chunk {i+1} ---\n")
    print(doc.page_content[:500])


 Chunks utilisés comme contexte :

--- Chunk 1 ---

5.5.3 (a), 5.5.3 (b), 5.5.3 (h), 5.18.3, 5.18.6 (e), 5.18.7, 5.20.1, 
8.1 9 November 
2016 
  
 
 
 
Guideline for good clinical practice E6(R2)    
EMA/CHMP/ICH/135/1995   Page 3/68 
 
 Guideline for good clinical practice E6(R2)  
Table of contents  
Introduction  ................................ ................................ ................................  6 
1. Glossary  ................................ ................................ ................................ .. 7 
2. The princ

--- Chunk 2 ---

these docu ments are no longer needed (see 4.9.4 and 5.5.12).   
 
 
 
Guideline for good clinical practice E6(R2)    
EMA/CHMP/ICH/135/1995   Page 36/68 
 
 The sponsor and the investigator/institution should sign the protocol, or an alternative document, to 
confirm this agreement.  
5.7.  Allocation of r esponsibilities  
Prior to initiating a trial, the sponsor should define, establish, and allocate all trial - relate d 

Architecture globale de l’app
[ PDF Upload ] → Extraction → Chunking → Embedding → FAISS

[ User Input ] → Recherche vectorielle → Gemini → Réponse

→ [ Affichage dans le chat Streamlit ]

6. Implement the Chat Interface -->
- Ouvre VS Code dans ton dossier projet: vérifie que ton fichier projet est bien présent avec le fichier app.py
cd /chemin/vers/projet_Gemini_PDF_Chatbot
- Active ton environnement virtuel :source venv/Scripts/activate  # ← sous Windows avec Git Bash
-Lance Streamlit avec app.py : streamlit run app.py